In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/imdb-reviews-pt-br.zip")

In [5]:
df.shape

(49459, 5)

In [6]:
df['label'] = df.sentiment.apply(lambda x: 0 if x =='neg' else 1)

In [7]:
df.tail(5)

,id,text_en,text_pt,sentiment,label
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1
49458,49460,Working-class romantic drama from director Mar...,O drama romântico da classe trabalhadora do di...,pos,1


# Testando o modelo com acuracia

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Divide os dados em treinamento e teste
text_train, text_test, labels_train, labels_test = train_test_split(text_data, labels, test_size=0.2)

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_train_vectors = vectorizer.fit_transform(text_train)
text_test_vectors = vectorizer.transform(text_test)

# Cria o classificador
clf = MultinomialNB()

# Treina o classificador
clf.fit(text_train_vectors, labels_train)

# Faz a previsão com os dados de teste
prediction = clf.predict(text_test_vectors)

# Calcula a precisão
accuracy = sum(prediction == labels_test) / len(labels_test)

print("A precisão é:", accuracy)

A precisão é: 0.850788515972503


In [9]:
# Dados de treinamento (texto e suas respectivas labels)
text_data = df.text_pt.values
labels = df.label.values

# Cria o objeto de vetorização de texto
vectorizer = CountVectorizer()

# Transforma o texto em um vetor de recursos
text_vectors = vectorizer.fit_transform(text_data)

# Cria o classificador
clf = MultinomialNB()

# Treina o classificador
clf.fit(text_vectors, labels)

MultinomialNB()

In [10]:
# Dados de teste (texto)
new_text = ["Hoje é um bom dia"]

# Transforma o novo texto em um vetor de recursos
new_text_vector = vectorizer.transform(new_text)

# Faz a previsão
prediction = clf.predict(new_text_vector)

print("A previsão é:", prediction[0])

A previsão é: 1


In [11]:
# Dados de teste (texto)
new_text = ["Hoje o dia está muito ruim"]

# Transforma o novo texto em um vetor de recursos
new_text_vector = vectorizer.transform(new_text)

# Faz a previsão
prediction = clf.predict(new_text_vector)

print("A previsão é:", prediction[0])

A previsão é: 0


In [12]:
import joblib
# Salva o modelo e o vetorizador
joblib.dump(clf, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']